# Importing Data

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 12
pd.options.display.max_columns = 10
pd.__version__

u'0.17.0'

We often times have a variety of input data.

- CSV
- Excel
- SQL
- JSON
- HDF5
- pickle
- msgpack
- Stata
- BigQuery

This is subset of the data from beeradvocate.com, via [Standford](https://snap.stanford.edu/data/web-RateBeer.html). It's strangely formatted.

<p style="font-size:20px"; style=font-family:Courier>
beer/name: Sausa Weizen<br>
beer/beerI: 47986<br>
beer/brewerId: 10325<br>
beer/ABV: 5.00<br>
beer/style: Hefeweizen<br>
review/appearance: 2.5<br>
review/aroma: 2<br>
review/time: 1234817823<br>
review/profileName: stcules<br>
review/text: A lot of foam. But a lot.	In the smell some banana, and then lactic and tart. Not a good start.	Quite dark orange in color, with a lively carbonation (now visible, under the foam).	Again tending to lactic sourness.	Same for the taste. With some yeast and banana.<br>
<br>
beer/name: Red Moon<br>
beer/beerId: 48213<br>
beer/brewerId: 10325<br>
beer/ABV: 6.20<br>
 ...<br>
</p>



The dataset was a bit large to processess all at once

```bash
$ wc -l beeradvocate.txt
 22212596 beeradvocate.txt
```

In [2]:
import toolz

def format_review(review):
    return dict(map(lambda x: x.strip().split(": ", 1), review))

def as_dataframe(reviews):
    col_names = {
        'beer/ABV': 'abv', 
        'beer/beerId': 'beer_id', 
        'beer/brewerId': 'brewer_id',
        'beer/name': 'beer_name', 
        'beer/style': 'beer_style',
        'review/appearance': 'review_appearance', 
        'review/aroma': 'review_aroma',
        'review/overall': 'review_overll', 
        'review/palate': 'review_palate',
        'review/profileName': 'profile_name', 
        'review/taste': 'review_taste',
        'review/text': 'text',
        'review/time': 'time'
    }
    df = pd.DataFrame(list(reviews))
    numeric = ['abv', 'review_appearance', 'review_aroma',
               'review_overall', 'review_palate', 'review_taste']
    df = (df.rename(columns=col_names)
            .replace('', np.nan))
    df[numeric] = df[numeric].astype(float)
    df['time'] = pd.to_datetime(df.time.astype(int), unit='s')
    return df

In [2]:
from toolz import partitionby, partition
def main():
    with open('beeradvocate.txt') as f:
        reviews = filter(lambda x: x != ('\n',),
                         partitionby(lambda x: x == '\n', f))
        reviews = map(format_review, reviews)
        reviews = partition(100000, reviews, pad=None)
        reviews = filter(None, reviews)
        os.makedirs('beer_reviews', exist_ok=True)
        for i, subset in enumerate(reviews):
            #print(i, end='\r')
            df = as_dataframe(subset)
            df.to_csv('beer_reviews/review_%s.csv' % i, index=False)
            

# Bootstrap

Let's get the data

In [4]:
df = pd.read_csv('data/beer2.csv.gz', 
                 index_col=0,
                 parse_dates=['time'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 13 columns):
abv                  48389 non-null float64
beer_id              50000 non-null int64
brewer_id            50000 non-null int64
beer_name            50000 non-null object
beer_style           50000 non-null object
review_appearance    50000 non-null float64
review_aroma         50000 non-null float64
review_overall       50000 non-null float64
review_palate        50000 non-null float64
profile_name         50000 non-null object
review_taste         50000 non-null float64
text                 49991 non-null object
time                 50000 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(2), object(4)
memory usage: 5.3+ MB


# CSV

http://pandas.pydata.org/pandas-docs/stable/io.html#csv-text-files

In [6]:
df.to_csv('data/beer.csv',index=False)

In [7]:
data = pd.read_csv('data/beer.csv', 
                    parse_dates=['time'])

# Excel

http://pandas.pydata.org/pandas-docs/stable/io.html#excel-files

In [8]:
# remove unicode
dfu = (pd.concat([df.select_dtypes(include=['object'])
                    .apply(lambda x: 
                               x.str.decode('ascii','xmlcharrefreplace')),
                  df.select_dtypes(exclude=['object'])],axis=1)
        .reindex(columns=df.columns))
dfu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 13 columns):
abv                  48389 non-null float64
beer_id              50000 non-null int64
brewer_id            50000 non-null int64
beer_name            47699 non-null object
beer_style           48370 non-null object
review_appearance    50000 non-null float64
review_aroma         50000 non-null float64
review_overall       50000 non-null float64
review_palate        50000 non-null float64
profile_name         50000 non-null object
review_taste         50000 non-null float64
text                 49991 non-null object
time                 50000 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(2), object(4)
memory usage: 5.3+ MB


In [9]:
dfu.to_excel('data/beer.xls',index=False)

In [10]:
data = pd.read_excel('data/beer.xls', sheetnames=[0])

# SQL

http://pandas.pydata.org/pandas-docs/stable/io.html#sql-queries

In [11]:
from sqlalchemy import create_engine
!rm -f data/beer.sqlite
engine = create_engine('sqlite:///data/beer.sqlite')

In [12]:
dfu.to_sql('table',engine)

In [13]:
data = pd.read_sql('table',engine)

# JSON

http://pandas.pydata.org/pandas-docs/stable/io.html#json

In [14]:
df.to_json('data/beer.json')

In [15]:
data = pd.read_json('data/beer.json')

# HDF

http://pandas.pydata.org/pandas-docs/stable/io.html#hdf5-pytables

In [16]:
# fixed format
dfu.to_hdf('data/beer.hdf',
           'df',
           mode='w',
           format='fixed')

/Users/jreback/miniconda/lib/python2.7/site-packages/pandas/core/generic.py:938: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['beer_name', 'beer_style', 'profile_name', 'text']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [17]:
data = pd.read_hdf('data/beer.hdf','df')

In [18]:
# wildly varying strings
df.text.str.len().describe()

count    49991.000000
mean       733.792003
std        392.219226
min         16.000000
25%        458.000000
50%        642.000000
75%        900.000000
max       4902.000000
Name: text, dtype: float64

# Timings

In [19]:
%timeit pd.read_excel('data/beer.xls', sheetnames=[0])

1 loops, best of 3: 2.62 s per loop


In [20]:
%timeit pd.read_sql('table', engine)

1 loops, best of 3: 757 ms per loop


In [21]:
%timeit pd.read_json('data/beer.json')

1 loops, best of 3: 1.17 s per loop


In [22]:
%timeit pd.read_csv('data/beer.csv', parse_dates=['time'])

1 loops, best of 3: 564 ms per loop


In [23]:
%timeit pd.read_hdf('data/beer.hdf','df')

1 loops, best of 3: 292 ms per loop


# Storing Text vs Data
http://matthewrocklin.com/blog/work/2015/03/16/Fast-Serialization/

# Operating on Large Data

In [24]:
chunks = pd.read_csv('data/beer2.csv.gz', 
                      index_col=0,
                      parse_dates=['time'],
                      chunksize=10000)
for i, chunk in enumerate(chunks):
    print("%d -> %d" % (i, len(chunk)))

0 -> 10000
1 -> 10000
2 -> 10000
3 -> 10000
4 -> 10000


# Using Odo
http://odo.readthedocs.org/

# Questions
- which formats provide fidelity
- which formats can you query/iterate
- which formats provide better interoprability
- which formats can you transmit over the wire
- which formats have better compression
- which formats allow multiple datasets in the same file